# Create layer for deforestation alerts

This notebook creates a point layer from the deforestation-alerts table in BigQuery, to be uploaded in Mapbox.
- 1. Download a copy of the existing table in BigQuery (deforestation-alerts/alerts).
- 2. Create a gdf with the lat and lon columns.
- 3. Convert to mbtiles without losing too many points at low zoom levels.

### Set up

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
from pathlib import Path

In [ ]:
# Generate mbtiles controling the number of points at low zoom levels to avoid losing them
def geojson_to_mbtiles(input_geojson, output_mbtiles,
                        min_zoom=0, max_zoom=14, layer_name="layer",
                        preserve_patterns=True, max_tile_features=1000):
    """
    Convert GeoJSON to MBTiles with automatic handling of feature density.

    Parameters:
    - preserve_patterns: If True, ensures global patterns are preserved (like keeping
                         points at low zoom levels).
    - max_tile_features: Maximum features per tile to prevent overcrowding.
    """
    input_geojson = Path(input_geojson)
    if not input_geojson.exists():
        raise FileNotFoundError(f"{input_geojson} does not exist.")
    
    command = [
        "tippecanoe",
        "-o", str(output_mbtiles),
        "-Z", str(min_zoom),
        "-z", str(max_zoom),
        "--layer", layer_name,
        "--maximum-tile-features", str(max_tile_features)
    ]

    if preserve_patterns:
        # These options help keep features visible at global zooms
        command.extend([
            "--drop-densest-as-needed",
            "--extend-zooms-if-still-dropping",
            "--base-zoom", str(max_zoom - 4),  # sensible default
            "--drop-rate", "0.5"
        ])
    else:
        # If not preserving patterns, use full detail at max zoom
        command.extend(["--full-detail", str(max_zoom)])

    command.append(str(input_geojson))
    
    print(f"Running: {' '.join(command)}")
    
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        if result.stdout:
            print(result.stdout)
        print("MBTiles generation complete!")
    except subprocess.CalledProcessError as e:
        print(f"Error ({e.returncode}): {' '.join(e.cmd)}")
        print(e.stderr or e.stdout or "No subprocess output.")
        raise

### Create layer

In [ ]:
# Read data
df = pd.read_csv('alerts_download_20260219.csv')

# Create geodataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf.crs = 'EPSG:4326'

# Save as geojson file
gdf.to_file('alerts_data.geojson', driver='GeoJSON')

In [ ]:
# Convert to mbtiles
geojson_to_mbtiles(
    "alerts_data.geojson",
    "alerts_data.mbtiles",
    min_zoom=0,
    max_zoom=14,
    layer_name="alerts_data",
    preserve_patterns=True
)